In [1]:
!git clone https://github.com/openai/multiagent-particle-envs.git
!pip install gym==0.10.5

Cloning into 'multiagent-particle-envs'...
remote: Enumerating objects: 242, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 242 (delta 0), reused 3 (delta 0), pack-reused 237
Receiving objects: 100% (242/242), 107.24 KiB | 3.57 MiB/s, done.
Resolving deltas: 100% (127/127), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 46.2 MB/s 
  Created wheel for gym: filename=gym-0.10.5-py3-none-any.whl size=1581307 sha256=ece4223d3c77cf680e36601a60a6cae2106a30b3ef96a8641fb1b958f30e4942
  Stored in directory: /root/.cache/pip/wheels/7a/2c/df/a05b548a40fae16ca400ecbeda0067e1a296499c1fbd7e0c9a
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2


In [2]:
cd multiagent-particle-envs

/content/multiagent-particle-envs


In [3]:
!pip install -e . 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/multiagent-particle-envs
  Running setup.py develop for multiagent


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim 
import numpy as np
from make_env import make_env

In [5]:
class Actor(nn.Module):
  def __init__(self,state_size,action_size,message_size):
    super(Actor,self).__init__()
    self.state_size = state_size
    self.action_size = action_size
    self.message_size = message_size
    self.fc2  = nn.Linear(32,64)
    self.fc3 = nn.Linear(64,128)
    self.fc4 = nn.Linear(128,256)
    self.fc5 = nn.Linear(256,512)
    self.fc6 = nn.Linear(512,256)
    self.fc7 = nn.Linear(256,128)
    self.fc8 = nn.Linear(128,64)
    self.fc9 = nn.Linear(64,32)
    self.fc10= nn.Linear(32,5)
  def forward(self,state,message):
    self.fc1 = nn.Linear(state.shape[0],32)
    self.fm1 = nn.Linear(message.shape[0],32)
    f1       = self.fc1(state)
    fm1      = self.fm1(message)
    cat      = torch.cat([f1.view(32,-1),fm1.view(32,-1)],0)
    cat      = torch.reshape(cat , (-1,))
    self.cat = nn.Linear(cat.shape[0],64)
    c1       = self.cat(cat)
    x        = self.fc5(f.relu(self.fc4(f.relu(self.fc3(f.relu(c1))))))
    x        = self.fc9(f.relu(self.fc8(f.relu(self.fc7(f.relu(self.fc6(f.relu(x))))))))
    x        = f.softmax(self.fc10(f.relu(x)))
    return x

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as f

class Critic(nn.Module):
  def __init__(self,state_size,action_size,message_size):
    super(Critic,self).__init__()
    self.state_size = state_size
    self.action_size = action_size
    self.message_size = message_size
    self.fc2  = nn.Linear(32,64)
    self.fc3 = nn.Linear(64,128)
    self.fc4 = nn.Linear(128,256)
    self.fc5 = nn.Linear(256,512)
    self.fc6 = nn.Linear(512,256)
    self.fc7 = nn.Linear(256,128)
    self.fc8 = nn.Linear(128,64)
    self.fc9 = nn.Linear(64,32)
    self.fc10= nn.Linear(32,1)
  def forward(self,state,action,reward):
    self.fc1 = nn.Linear(state.shape[0],32)
    self.fa1 = nn.Linear(action.shape[0],32)
    self.fr1 = nn.Linear(2,32)
    f1       = self.fc1(state)
    fa1      = self.fa1(action)
    fr1      = self.fr1(reward)
    cat      = torch.cat([f1.view(32,-1),fa1.view(32,-1),fr1.view(32,-1)],0)
    cat      = torch.reshape(cat , (-1,))
    self.cat = nn.Linear(cat.shape[0],64)
    c1       = self.cat(cat)
    x        = self.fc5(f.relu(self.fc4(f.relu(self.fc3(f.relu(c1))))))
    x        = self.fc9(f.relu(self.fc8(f.relu(self.fc7(f.relu(self.fc6(f.relu(x))))))))
    x        = self.fc10(f.relu(x))
    return x


In [39]:
class MGN(nn.Module):
  def __init__(self,state_size,action_size,message_size):
    super(MGN,self).__init__()
    #H Length = 3
    self.state_size,self.action_size,self.message_size = state_size,action_size,message_size
    self.l2  = nn.Linear(128,512)
    self.l3  = nn.Linear(512,128)
    self.l4  = nn.Linear(128,64)
    self.l5  = nn.Linear(64,32)
    self.l6  = nn.Linear(32,5)
  def action_predictor(self,observation):
    self.fa1 = nn.Linear(observation.shape[0],32)
    self.fa2 = nn.Linear(32,64)
    self.fa3 = nn.Linear(64,128)
    self.fa4 = nn.Linear(128,64)
    self.fa5 = nn.Linear(64,32)
    self.out = nn.Linear(32,5)
    x        = self.fa5(f.relu(self.fa4(f.relu(self.fa3(self.fa2(f.relu(self.fa1(observation))))))))
    x       = f.softmax(self.out(x))
    #status:- al bugs cleared
    return x
  def observation_predictor(self,observation,action_t1,action_t2):
    self.fo1 = nn.Linear(observation.shape[0],32)
    self.fat1= nn.Linear(action_t1.shape[0],32)
    self.fat2= nn.Linear(action_t2.shape[0],32)
    o = self.fo1(observation)
    a1 = self.fat1(action_t1)
    a2 = self.fat2(action_t2)
    cat = torch.cat([o.view(32,-1),a1.view(32,-1),a2.view(32,-1)],1)
    cat = torch.reshape(cat,(-1,))
    self.fat3 = nn.Linear(cat.shape[0],128)
    self.fat4 = nn.Linear(128,512)
    self.fat5 = nn.Linear(512,128)
    self.fat6 = nn.Linear(128,64)
    self.fat7 = nn.Linear(64,32)
    self.fat8 = nn.Linear(32,observation.shape[0])
    x = self.fat7(f.relu(self.fat6(f.relu(self.fat5(f.relu(self.fat4(f.relu(self.fat3(cat)))))))))
    x = self.fat8(f.relu(x))
    #status = all bugs cleard
    return x
  def policy(self,observation,message):
    self.lo1 = nn.Linear(observation.shape[0],32)
    self.lm1 = nn.Linear(message.shape[0],32)
    o        = self.lo1(observation)
    m        = self.lm1(message)
    cat      = torch.cat([o.view(32,-1),m.view(32,-1)],1)
    cat      = torch.reshape(cat,(-1,))
    self.l1  = nn.Linear(cat.shape[0],128)
    x = self.l5(f.relu(self.l4(f.relu(self.l3(f.relu(self.l2(f.relu(self.l1(cat)))))))))
    x = f.softmax(self.l6(x))
    #status = all bugs cleard
    return x
  def ITGM1(self,message,observation,action):
    #passing action at time step t
    #passing observation at time step t
    #passing message at time step t-1
    #implemnting the observation predictor
    action_predictor = self.action_predictor(action)
    observation_predictor = self.observation_predictor(observation,action,action_predictor)
    #implementing the action_predictor
    policy = self.policy(observation_predictor,message)    
    return message,torch.cat([observation_predictor,policy],0)
  def attention(self,message,trajectory):
    self.wq = nn.Linear(message.shape[0],32)
    self.wk = nn.Linear(trajectory.shape[0],32)
    self.wv = nn.Linear(trajectory.shape[0],32)
    q       = self.wq(message)
    k       = self.wk(trajectory)
    v       = self.wv(trajectory)
    mat     = torch.matmul(q,k)
    soft    = f.softmax(mat)
    soft    = torch.tensor([soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft,soft])
    msg     = torch.matmul(soft,v)
    #status = all bugs cleard
    return msg
  def forward(self,message,observation,action):
    msg,trajectory = self.ITGM1(message,observation,action)
    msg            = self.attention(msg,trajectory)
    return msg


In [40]:
env = make_env("simple_reference")

In [41]:
state = env.reset()
obs = []
acts = []
a = Actor(21,5,5)
for i in range(len(state)):
  o = torch.from_numpy(state[i]).float()
  obs.append(o)
  act = a(o,torch.zeros([0*5]))
  acts.append(act)

/usr/local/lib/python3.7/dist-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [42]:
r = torch.tensor([0.1232,0],dtype = torch.float32)

In [43]:
c = Critic(21,5,5)

In [34]:
c(obs[0],act[0],r)

RuntimeError: ignored

In [36]:
l = nn.Linear(21,32)
m = nn.Linear(5,32)
k = nn.Linear(2,32)

In [37]:
a1 = l(obs[0])
a2 = m(acts[0])
a3 = k(r)

In [31]:
k

Linear(in_features=1, out_features=32, bias=True)